## Sliding window for feature extraction

In [1]:
# Load data
# TODO we may need to re-separate the data into original files

from lisa.config import INTERIM_DATA_DIR
import polars as pl

df = pl.read_csv(INTERIM_DATA_DIR / "pilot_data.csv")

2024-08-01 15:54:29.630 | INFO     | lisa.config:<module>:15 - PROJ_ROOT path is: /Users/tomwilson/code/LISA


In [2]:
df

Electric Potential.1,D1_Global Angle.x,D1_Global Angle.y,D1_Global Angle.z,D1_HighG.x,D1_HighG.y,D1_HighG.z,D1_accel.x,D1_accel.y,D1_accel.z,D1_gyro.x,D1_gyro.y,D1_gyro.z,D1_mag.x,D1_mag.y,D1_mag.z,D2_Global Angle.x,D2_Global Angle.y,D2_Global Angle.z,D2_HighG.x,D2_HighG.y,D2_HighG.z,D2_accel.x,D2_accel.y,D2_accel.z,D2_gyro.x,D2_gyro.y,D2_gyro.z,D2_mag.x,D2_mag.y,D2_mag.z,D3_Global Angle.x,D3_Global Angle.y,D3_Global Angle.z,D3_HighG.x,D3_HighG.y,D3_HighG.z,…,D5_gyro.z,D5_mag.x,D5_mag.y,D5_mag.z,D6_Global Angle.x,D6_Global Angle.y,D6_Global Angle.z,D6_HighG.x,D6_HighG.y,D6_HighG.z,D6_accel.x,D6_accel.y,D6_accel.z,D6_gyro.x,D6_gyro.y,D6_gyro.z,D6_mag.x,D6_mag.y,D6_mag.z,D7_Global Angle.x,D7_Global Angle.y,D7_Global Angle.z,D7_HighG.x,D7_HighG.y,D7_HighG.z,D7_accel.x,D7_accel.y,D7_accel.z,D7_gyro.x,D7_gyro.y,D7_gyro.z,D7_mag.x,D7_mag.y,D7_mag.z,ACTIVITY,TIME,TRIAL
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,i64,i64
3.131464,2.257107,0.021965,1.961349,12.916076,4.066172,10.763396,9.356541,-0.857124,-1.098939,-1.188007,-0.150697,-1.811033,-0.000017,0.000014,0.000005,2.327128,0.058121,2.065824,5.979665,3.109426,-0.239187,9.514557,-3.660734,-1.910573,-1.339237,0.363165,-1.159252,-0.000036,-0.000002,-0.000032,-1.230088,0.088267,-0.717407,1.674306,-5.501291,13.394449,…,-7.013557,-0.00007,0.000023,-0.000008,0.827088,-1.046932,1.374667,8.371531,5.022918,0.0,7.517794,-0.792481,1.149217,-0.643793,-0.025027,-0.041535,-0.000044,-0.000007,0.000042,-0.443208,-1.639701,-0.324574,12.916076,-3.109426,0.71756,10.122684,-8.62152,-3.761291,0.141645,1.775887,-6.670628,-0.000041,-0.000028,-0.000018,"""walk""",0,0
3.130854,2.255623,0.022602,1.959735,12.868238,3.994416,10.643803,9.407297,-0.951456,-1.162385,-1.187528,-0.142124,-1.802672,-0.000017,0.000014,0.000005,2.325831,0.057856,2.065244,5.716559,2.894158,0.023919,9.535626,-3.789303,-1.832522,-1.328108,0.357414,-1.169956,-0.000036,-0.000002,-0.000032,-1.230326,0.087594,-0.717567,1.746062,-5.52521,13.203099,…,-7.013717,-0.00007,0.000023,-0.000008,0.826731,-1.047273,1.374304,8.347612,4.903325,0.0,7.533116,-0.810916,1.130542,-0.648745,-0.029447,-0.043345,-0.000044,-0.000007,0.000042,-0.437664,-1.639763,-0.328927,12.963913,-3.085507,0.837153,10.204566,-8.503486,-3.630328,0.233022,1.775089,-6.670681,-0.000041,-0.000028,-0.000018,"""walk""",1,0
3.118952,2.254139,0.023239,1.958121,12.820401,3.92266,10.52421,9.458055,-1.045787,-1.225831,-1.187049,-0.133551,-1.794312,-0.000017,0.000014,0.000005,2.324534,0.057592,2.064664,5.453454,2.67889,0.287024,9.556695,-3.917872,-1.754471,-1.316979,0.351663,-1.180659,-0.000036,-0.000002,-0.000032,-1.230563,0.086922,-0.717726,1.817818,-5.549129,13.01175,…,-7.013877,-0.00007,0.000023,-0.000008,0.826374,-1.047614,1.373941,8.323693,4.783731,0.0,7.548439,-0.829351,1.111867,-0.653697,-0.033867,-0.045156,-0.000044,-0.000007,0.000042,-0.432118,-1.639814,-0.333277,13.01175,-3.061588,0.956746,10.286448,-8.385452,-3.499365,0.324399,1.77429,-6.670734,-0.000041,-0.000028,-0.000018,"""walk""",2,0
3.168391,2.252655,0.023875,1.956507,12.772564,3.850904,10.404616,9.508811,-1.140119,-1.289278,-1.18657,-0.124978,-1.785952,-0.000017,0.000014,0.000005,2.323237,0.057327,2.064084,5.190349,2.463622,0.550129,9.577765,-4.04644,-1.67642,-1.30585,0.345912,-1.191362,-0.000036,-0.000002,-0.000032,-1.230801,0.08625,-0.717886,1.889574,-5.573048,12.820401,…,-7.014037,-0.000069,0.000023,-0.000008,0.826017,-1.047956,1.373578,8.299774,4.664138,0.0,7.563762,-0.847787,1.093192,-0.658649,-0.038287,-0.046966,-0.000044,-0.000007,0.000042,-0.42657,-1.639852,-0.337624,13.059587,-3.03767,1.07634,10.36833,-8.267418,-3.368402,0.415776,1.773491,-6.670787,-0.000041,-0.000027,-0.000018,"""walk""",3,0
3.146418,2.25117,0.02451,1.954893,12.724727,3.779148,10.285023,9.559568,-1.2344

In [3]:
# sliding window

rolling = df.rolling(index_column="TIME", period="300i", group_by="TRIAL")
rdf = rolling.agg(
        [

        pl.when(pl.len() >= 5).then(pl.mean("Electric Potential.1")).alias("mean_a"),

        pl.when(pl.len() >= 5).then(pl.min("Electric Potential.1")).alias("min_a"),

        pl.when(pl.len() >= 5).then(pl.max("Electric Potential.1")).alias("max_a"),

    ]
)

In [55]:
# sliding window
def sliding_window(df, period=300):
    """
    Apply sliding window aggregation on a DataFrame.

    Args:
        df (polars.DataFrame): The input DataFrame.
        period (int): The window size in number of rows. Default is 300.

    Returns:
        polars.DataFrame: The aggregated DataFrame with rolling window statistics.

    """
    # List of columns to exclude from aggregation
    exclude_columns = ["TIME", "TRIAL", "ACTIVITY"]

    # Get the list of columns to aggregate
    columns_to_aggregate = [col for col in df.columns if col not in exclude_columns]

    # Define the rolling window
    rolling = df.rolling(index_column="TIME", period=f"{period}i", group_by="TRIAL")

    # Define the aggregation operations
    aggregations = []
    for col in columns_to_aggregate:
        aggregations.extend([
            pl.first(col).alias(f"first_{col}"),
            pl.last(col).alias(f"last_{col}"),
            pl.max(col).alias(f"max_{col}"),
            pl.min(col).alias(f"min_{col}"),
            pl.mean(col).alias(f"mean_{col}"),
            pl.std(col).alias(f"std_{col}"),
        ])

    # Perform the aggregation
    result = rolling.agg(aggregations)

    # Check if TIME resets to 0 when TRIAL increases by 1
    trial_check = result.with_columns((pl.col("TRIAL") - pl.col("TRIAL").shift(1)).alias("TRIAL_INCREASE"))
    time_resets_correctly = trial_check.filter(pl.col("TRIAL_INCREASE") == 1)["TIME"].to_list() == [0] * len(trial_check.filter(pl.col("TRIAL_INCREASE") == 1))

    # Remove rows before first 'full' window
    if time_resets_correctly:
        result = result.filter(pl.col("TIME") > period-2)
    else:
        raise ValueError("Time does not reset to 0 when TRIAL increases by 1. Unable to remove rows before first full window.")


    # Add the ACTIVITY column back in by matching TRIAL
    # Assumes every trial has one activity type
    activity_map = dict(zip(df["TRIAL"], df["ACTIVITY"]))
    result = result.with_columns(ACTIVITY=pl.col("TRIAL").replace_strict(activity_map))
    
    return result


In [54]:
import numpy as np
import pytest
from polars.testing import assert_frame_equal

def test_sliding_window():
    # Create a sample DataFrame
    df = pl.DataFrame({
        "TRIAL": [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        "TIME": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
        "ACTIVITY": ["A", "A", "A", "A", "A", "A", "A", "A", "A", "A"],
        "Value": [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    }, strict=False)

    # Call the sliding_window function
    result= sliding_window(df, period=3)

    # Check the result
    expected_result = pl.DataFrame({
        "TRIAL": [0, 0, 0, 0, 0, 0, 0, 0],
        "TIME": [2, 3, 4, 5, 6, 7, 8, 9],
        "ACTIVITY": ["A", "A", "A", "A", "A", "A", "A", "A"],
        "first_Value": [10, 20, 30, 40, 50, 60, 70, 80],
        "last_Value": [30, 40, 50, 60, 70, 80, 90, 100],
        "max_Value": [30, 40, 50, 60, 70, 80, 90, 100],
        "min_Value": [10, 20, 30, 40, 50, 60, 70, 80],
        "mean_Value": [20, 30, 40, 50, 60, 70, 80, 90],
        "std_Value": [10, 10, 10, 10, 10, 10, 10, 10],
    }, strict=False)
    print(result)
    assert_frame_equal(result, expected_result, check_column_order=False, check_dtypes=False)

    # # Check the dropped rows
    # expected_dropped_rows = pd.DataFrame({
    #     "TIME": [6, 7, 8, 9, 10],
    #     "TRIAL": [1, 1, 1, 1, 1],
    #     "ACTIVITY": ["B", "B", "B", "B", "B"],
    #     "Value1": [60, 70, 80, 90, 100],
    #     "Value2": [6, 7, 8, 9, 10]
    # })

    # assert dropped_rows.equals(expected_dropped_rows)

test_sliding_window()

shape: (8, 9)
┌───────┬──────┬─────────────┬────────────┬───┬───────────┬────────────┬───────────┬──────────┐
│ TRIAL ┆ TIME ┆ first_Value ┆ last_Value ┆ … ┆ min_Value ┆ mean_Value ┆ std_Value ┆ ACTIVITY │
│ ---   ┆ ---  ┆ ---         ┆ ---        ┆   ┆ ---       ┆ ---        ┆ ---       ┆ ---      │
│ i64   ┆ i64  ┆ i64         ┆ i64        ┆   ┆ i64       ┆ f64        ┆ f64       ┆ str      │
╞═══════╪══════╪═════════════╪════════════╪═══╪═══════════╪════════════╪═══════════╪══════════╡
│ 0     ┆ 2    ┆ 10          ┆ 30         ┆ … ┆ 10        ┆ 20.0       ┆ 10.0      ┆ A        │
│ 0     ┆ 3    ┆ 20          ┆ 40         ┆ … ┆ 20        ┆ 30.0       ┆ 10.0      ┆ A        │
│ 0     ┆ 4    ┆ 30          ┆ 50         ┆ … ┆ 30        ┆ 40.0       ┆ 10.0      ┆ A        │
│ 0     ┆ 5    ┆ 40          ┆ 60         ┆ … ┆ 40        ┆ 50.0       ┆ 10.0      ┆ A        │
│ 0     ┆ 6    ┆ 50          ┆ 70         ┆ … ┆ 50        ┆ 60.0       ┆ 10.0      ┆ A        │
│ 0     ┆ 7    ┆ 60       

In [26]:
# Create the first DataFrame (df1) with 'trial' and 'activity' columns
df1 = pl.DataFrame({
    "TRIAL": [0, 0, 1, 1, 2],
    "ACTIVITY": ["a", "a", "b", "b", "a"]
})

# Create the second DataFrame (df2) with only the 'trial' column
df2 = pl.DataFrame({
    "TRIAL": [1,1, 2, 0,2]
})

df1_dict = dict(zip(df1["TRIAL"], df1["ACTIVITY"]))

expected_result = pl.DataFrame({
    "TRIAL": [1,1, 2, 0,2],
    "ACTIVITY": ["b","b", "a", "a", "a"]
})

# Perform the join operation
# result = df2.with_columns(pl.Series("activity", df2["trial"].apply(lambda x: df1_dict.get(x))))
# result = df2.with_columns(df1_dict[pl.col("TRIAL")])
# Display the result

result = df2.with_columns(ACTIVITY=pl.col("TRIAL").replace_strict(df1_dict))
print(result==expected_result)

shape: (5, 2)
┌───────┬──────────┐
│ TRIAL ┆ ACTIVITY │
│ ---   ┆ ---      │
│ bool  ┆ bool     │
╞═══════╪══════════╡
│ true  ┆ true     │
│ true  ┆ true     │
│ true  ┆ true     │
│ true  ┆ true     │
│ true  ┆ true     │
└───────┴──────────┘


In [56]:
rdf=sliding_window(df)

In [57]:
rdf

TRIAL,TIME,first_Electric Potential.1,last_Electric Potential.1,max_Electric Potential.1,min_Electric Potential.1,mean_Electric Potential.1,std_Electric Potential.1,first_D1_Global Angle.x,last_D1_Global Angle.x,max_D1_Global Angle.x,min_D1_Global Angle.x,mean_D1_Global Angle.x,std_D1_Global Angle.x,first_D1_Global Angle.y,last_D1_Global Angle.y,max_D1_Global Angle.y,min_D1_Global Angle.y,mean_D1_Global Angle.y,std_D1_Global Angle.y,first_D1_Global Angle.z,last_D1_Global Angle.z,max_D1_Global Angle.z,min_D1_Global Angle.z,mean_D1_Global Angle.z,std_D1_Global Angle.z,first_D1_HighG.x,last_D1_HighG.x,max_D1_HighG.x,min_D1_HighG.x,mean_D1_HighG.x,std_D1_HighG.x,first_D1_HighG.y,last_D1_HighG.y,max_D1_HighG.y,min_D1_HighG.y,mean_D1_HighG.y,…,first_D7_gyro.x,last_D7_gyro.x,max_D7_gyro.x,min_D7_gyro.x,mean_D7_gyro.x,std_D7_gyro.x,first_D7_gyro.y,last_D7_gyro.y,max_D7_gyro.y,min_D7_gyro.y,mean_D7_gyro.y,std_D7_gyro.y,first_D7_gyro.z,last_D7_gyro.z,max_D7_gyro.z,min_D7_gyro.z,mean_D7_gyro.z,std_D7_gyro.z,first_D7_mag.x,last_D7_mag.x,max_D7_mag.x,min_D7_mag.x,mean_D7_mag.x,std_D7_mag.x,first_D7_mag.y,last_D7_mag.y,max_D7_mag.y,min_D7_mag.y,mean_D7_mag.y,std_D7_mag.y,first_D7_mag.z,last_D7_mag.z,max_D7_mag.z,min_D7_mag.z,mean_D7_mag.z,std_D7_mag.z,ACTIVITY
i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
0,299,3.131464,0.297907,3.785151,0.27746,2.581347,1.445571,2.257107,2.133543,2.257107,2.133543,2.186005,0.025971,0.021965,-0.530777,0.097351,-0.530777,-0.021068,0.160638,1.961349,2.23842,2.23842,1.873504,2.017557,0.135867,12.916076,13.346611,25.353779,11.95933,16.821993,3.303436,4.066172,1.937411,11.002583,-3.826985,1.741278,…,0.141645,0.276847,4.011855,-6.30054,-0.359595,2.039678,1.775887,-0.308317,1.775887,-1.49526,-0.183394,0.780013,-6.670628,1.059515,3.692355,-6.67116,-0.020432,3.480854,-0.000041,-0.000044,-0.000041,-0.000046,-0.000044,0.000001,-0.000028,-0.000021,-0.000007,-0.000028,-0.000015,0.000006,-0.000018,-0.000018,-0.000012,-0.000018,-0.000015,0.000002,"""walk"""
0,300,3.130854,0.290278,3.785151,0.27746,2.571877,1.45125,2.255623,2.133151,2.255623,2.133151,2.185592,0.025822,0.022602,-0.537833,0.097351,-0.537833,-0.022934,0.163365,1.959735,2.238629,2.238629,1.873504,2.018482,0.136425,12.868238,13.394449,25.353779,11.95933,16.823587,3.30166,3.994416,2.152679,11.002583,-3.826985,1.7349,…,0.233022,0.124072,4.011855,-6.30054,-0.359653,2.039663,1.775089,-0.305655,1.775089,-1.49526,-0.190332,0.77174,-6.670681,1.062337,3.692355,-6.67116,0.005344,3.460013,-0.000041,-0.000044,-0.000041,-0.000046,-0.000044,0.000001,-0.000028,-0.000021,-0.000007,-0.000028,-0.000015,0.000006,-0.000018,-0.000018,-0.000012,-0.000018,-0.000015,0.000002,"""walk"""
0,301,3.118952,0.283259,3.785151,0.27746,2.562385,1.456884,2.254139,2.132746,2.254139,2.132746,2.185183,0.025682,0.023239,-0.544891,0.097351,-0.544891,-0.024826,0.166098,1.958121,2.238827,2.238827,1.873504,2.019412,0.136974,12.820401,13.442286,25.353779,11.95933,16.825501,3.299525,3.92266,2.367947,11.002583,-3.826985,1.729478,…,0.324399,-0.028702,4.011855,-6.30054,-0.360526,2.039465,1.77429,-0.302992,1.77429,-1.49526,-0.197259,0.763321,-6.670734,1.06516,3.692355,-6.67116,0.031131,3.438854,-0.000041,-0.000044,-0.000041,-0.000046,-0.000044,0.000001,-0.000028,-0.000021,-0.000007,-0.000028,-0.000015,0.000006,-0.000018,-0.000018,-0.000012,-0.000018,-0.000015,0.000002,"""walk"""
0,302,3.168391,0.284785,3.785151,0.27746,2.552937,1.462441,2.252655,2.132331,2.252655,2.132331,2.184777,0.02555,0.023875,-0.551951,0.097351,-0.551951,-0.026743,0.168839,1.956507,2.239013,2.239013,1.873504,2.020348,0.137512,12.772564,13.490124,25.353779,11.95933,16.827733,3.297032,3.850904,2.583215,11.002583,-3.826985,1.725014,…,0.415776,-0.181476,4.011855,-6.30054,-0.362212,2.039106,1.7

In [ ]:
(624995-617520)

7475

In [ ]:
shape: (625_852, 639)

In [ ]:
299*25

7475

In [ ]:
# null rows in trial 0
474

In [ ]:
rdf['mean_a'].is_null().sum()

ColumnNotFoundError: mean_a

In [ ]:
rdf.shape[0] /(rdf.shape[0] - rdf['mean_a'].is_null().sum())

24999.8